In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, date2num
import matplotlib.ticker as ticker
import geopandas
from shapely.geometry import shape
from PIL import Image
import glob
from IPython.display import HTML

# Load data from nytimes
# Has data for each county in each state
df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv', 
                 parse_dates=['date'])

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    
    
info = dotdict({'name': "Georgia", 
              'json': "GA-13-georgia-counties",
              'county': "Fulton",
              'abr': "GA"
             })

info = dotdict({'name': "New York", 
              'json': "NY-36-new-york-counties",
              'county': "New York City",
              'abr': "NY"
             })

info = dotdict({'name': "North Carolina", 
              'json': "NC-37-north-carolina-counties",
              'county': "Wake",
              'abr': "NC"
             })

In [ ]:
# Print the data
df.head()

In [ ]:
# Select South Carolina
state = df[df.state == info.name]
# Group by county for each date
state = state.pivot(index='date', columns='county', values='cases')
state = state.fillna(0.0)
state.tail(5)

In [ ]:
plot = state.plot(figsize=(12,8), linewidth=5, cmap='PuBu')
plot.legend(ncol=2, bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

In [ ]:
# Copy data
new_per_day = state.copy(deep=True)

# Get difference per day instead of total cases
for county in state.columns:
    new_per_day[county] = new_per_day[county].diff()
new_per_day = new_per_day.fillna(0.0)
new_per_day.tail(5) 


In [ ]:
# Plot again using matplotib
fig, ax = plt.subplots(figsize=(16,9))

# Average over the last two weeks
rolling_days = 14

dates = date2num(state.index)
# This time we do rolling mean to take some of the jittering out
ax.plot_date(dates, new_per_day[info.county].rolling(rolling_days).mean(), fmt='-', linewidth=5)

# Plot trend line for the last set of days averaged over
z = np.polyfit(dates[-rolling_days:], new_per_day[info.county].rolling(rolling_days).mean()[-rolling_days:], 1)
p = np.poly1d(z)
ax.plot(dates[-rolling_days:],p(dates[-rolling_days:]),'--', linewidth=3)

date_form = DateFormatter("%b-%d")
ax.xaxis.set_major_formatter(date_form)
#ax.set_yscale('log')
plt.show()

In [ ]:
# Get geometry data
# https://github.com/deldersveld/topojson
geodata = geopandas.read_file(f"https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/{info.json}.json")

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
geodata.plot(color="white", edgecolor='black', ax=ax)
max_count = np.max(state.max(axis=0))

geodata["color"] = np.zeros_like(geodata.NAME)

for c in state.iteritems():
    geodata.loc[geodata.NAME == c[0], 'color'] = c[1][-1]/max_count

geodata.plot(column='color', ax=ax, cmap='PuBu')
ax.set_title(f"Current Cases in {info.abr}")
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.get_yaxis().set_visible(False)
ax.get_xaxis().set_visible(False)
plt.show()
plt.close()

In [ ]:
# Get max from data and max of all the counties
max_count = np.max(state.T.max(axis=0))
total = state.T.sum()

# Count is used to save images
count = 0

os.mkdir(f"{info.abr}_gif") if not os.path.isdir(f"{info.abr}_gif") else None

for date, row in state.T.iteritems():
    fig, ax = plt.subplots(figsize=(16,9))
    geodata.plot(color='white',ax=ax, edgecolor='k')
    for c in row.iteritems():
        color = c[1]/max_count
        geodata.loc[geodata.NAME == c[0], 'color'] = color
    
    geodata.plot(column='color', ax=ax, cmap='PuBu')

    ax.set_title(f"{date.month_name()} {date.day}\nTotal Cases: {int(total[count])}")
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.get_yaxis().set_visible(False)
    ax.get_xaxis().set_visible(False)
    count += 1
    fig.savefig(f"{info.abr}_gif/{count:03}.png")
    plt.close()

In [ ]:
# Create the frames
frames = []
imgs = glob.glob(f"{info.abr}_gif/*.png")
imgs = np.sort(imgs)

for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

# Pause on the last frame for longer
for i in range(0,20):
    frames.append(Image.open(imgs[-1]))

# Save into a GIF file that loops forever
frames[0].save(f'cases_in_{info.abr}.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=600, loop=0)


In [ ]:
# Show gif in jupyter
HTML(f'<img src="cases_in_{info.abr}.gif">')